<a href="https://colab.research.google.com/github/RaulCapellanFernandez/AlgoritmosOptimizacion/blob/main/ActGuiada2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Actividad Guiada 2: Algoritmos de optimización.

- Autor: Raúl Capellán Fernández
-DNI:    71471266P
-Fecha:  26 / 12 / 2020